In [1]:
# Load sql magic extension
%load_ext sql
# Connect to database
%sql mysql://root@127.0.0.1/current
# Import python modules
import numpy as np
import pandas as pd

u'Connected: root@current'

### Define functions

In [36]:
def exec_statement(string):
    ''' Accepts a string and executes it as a sql command '''
    %sql SET @query = :string;
    %sql PREPARE stmt FROM @query;
    return_val = %sql EXECUTE stmt;
    %sql DEALLOCATE PREPARE stmt;
    return return_val


def df_crossjoin(df1, df2, **kwargs):
    ''' Cross (or Cartesian)-joins two pandas dataframes '''
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1

    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey', axis=1)
    res.index = pd.MultiIndex.from_product((df1.index, df2.index))

    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)

    return res

### Define linking variables

In [3]:
tableA = 'patient_discharges'
tableB = 'births'

Define fields to compare tables by

In [4]:
tableA_fields = ['date_of_birth'] #pdd table
tableB_fields = ['mothers_date_of_birth'] #bc table

#comparisons = [
#    "date_of_birth = mothers_date_of_birth",
#    "delivered_on = date_of_delivery"
#]

Define field(s) to block by

In [5]:
tableA_blocks = ['hospital_id','delivered_on']
tableB_blocks = ['hospital_id','date_of_delivery']

### Blocking
Find all possible values for 1st blocking field ('hospital_id')

In [7]:
bl_stmt = 'SELECT T.blks FROM (' + \
'SELECT DISTINCT ' + tableA_blocks[0] + ' AS blks ' \
'FROM ' +  tableA + ' UNION ' + \
'SELECT DISTINCT ' + tableB_blocks[0] + ' AS blks ' \
'FROM ' +  tableB + ') as T ORDER BY blks;'

block_list1 = exec_statement(bl_stmt);

0 rows affected.
0 rows affected.
12 rows affected.
0 rows affected.
+------+
| blks |
+------+
|  1   |
|  2   |
|  3   |
|  4   |
|  6   |
|  7   |
|  8   |
|  9   |
|  10  |
|  11  |
|  12  |
|  13  |
+------+


Find all possible values for 2nd blocking field (delivery date)

In [8]:
bl_stmt = 'SELECT DISTINCT ' + tableB_blocks[1] + \
' FROM ' +  tableB + ' ORDER BY ' + tableB_blocks[1] + ';'

block_list2 = exec_statement(bl_stmt);

0 rows affected.
0 rows affected.
1554 rows affected.
0 rows affected.


In [41]:
linked_match_count = [0]*len(tableA_fields)
match_count = [0]*len(tableA_fields)
pair_count = 0

### Craft each block

In [42]:
stmt1 = 'SELECT ' + ",".join(tableA_fields) + ' FROM ' + tableA + ' a ' + \
'JOIN deliveries b ON a.id=b.patient_discharge_id ' + \
'WHERE a.' + tableA_blocks[0] + '=' + repr(str(block_list1[0][0])) + \
'AND a.admitted_on<' + repr(str(block_list2[500][0])) + \
'AND a.discharged_on>' + repr(str(block_list2[500][0])) + ';'
print(stmt1)
stmt2 = 'SELECT ' + ",".join(tableB_fields) + ' FROM ' + tableB + ' b ' + \
'WHERE b.' + tableB_blocks[0] + '=' + repr(str(block_list1[0][0])) + \
'AND b.' + tableB_blocks[1] + '=' + repr(str(block_list2[500][0])) + ';'
print('\n' + stmt2)
blockA = exec_statement(stmt1)
blockA_df = blockA.DataFrame()
blockB = exec_statement(stmt2)
blockB_df = blockB.DataFrame()

SELECT date_of_birth FROM patient_discharges a JOIN deliveries b ON a.id=b.patient_discharge_id WHERE a.hospital_id='1'AND a.admitted_on<'2012-05-11'AND a.discharged_on>'2012-05-11';

SELECT mothers_date_of_birth FROM births b WHERE b.hospital_id='1'AND b.date_of_delivery='2012-05-11';
0 rows affected.
0 rows affected.
17 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
10 rows affected.
0 rows affected.


### Cross join fields-of-interest from both blocks

In [43]:
cross_table = df_crossjoin(blockA_df, blockB_df)

# Count field matches and dump each into new column of dataframe
bool_table = pd.DataFrame(columns=tableA_fields)
for index,(fieldA, fieldB) in enumerate(zip(tableA_fields,tableB_fields)):
    bool_table[fieldA] = np.where(cross_table[fieldA]==cross_table[fieldB],1,0)

match_count += bool_table.sum()
pair_count += bool_table.count()[0]

In [34]:
np.where(cross_table[tableA_fields[0]]==cross_table[tableB_fields[0]],1,0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])